In [1]:
import os
import math
from docplex.mp.model import Model
from docplex.mp.relax_linear import LinearRelaxer

def fill_variables(file, n_customers, n_available_depots, coordinates_depots, coordinates_customers, v_capacity, capacities_depots,
        customers_demands, opening_costs_depots, opening_cost_route, zero_or_not):
    
    line = next(file)
    n_customers = int(line.strip())
    print("Número de customers: " + str(n_customers))
    line = next(file)
    n_available_depots = int(line.strip())
    print("Número de depots disponibles: " + str(n_available_depots))
    line = next(file)
    line = next(file)
            
    #coordenadas depots
    while(line.strip()):
        coordinates_depots.append(line.split())    
        line = next(file)
            
    #print("Coordenadas depots:")
    #print(coordinates_depots)
            
    line = next(file)
            
        #Coordenadas customers
    while(line.strip()):
        coordinates_customers.append(line.split())
                
        line = next(file)
            
    #print("Coordenadas customers:")
    #print(coordinates_customers)
            
    line = next(file)
            
    #Capacidad de vehículos
    v_capacity = int(line.strip())
    #print("Capacidad vehículos: " + v_capacity)
    line = next(file)
    line = next(file)
            
    #Capacidad de los depots
    while(line.strip()):
        capacities_depots.append(line.split())
                
        line = next(file)
            
    #print("Capacidades depots:")
    #print(capacities_depots)
            
    line = next(file)
            
    while(line.strip()):
        customers_demands.append(line.strip())
                
        line = next(file)
            
    #print("Demandas customers:")
    #print(customers_demands)
            
    line = next(file)
            
    while(line.strip()):
        opening_costs_depots.append(line.split())
                
        line = next(file)
            
    #print("Costos depots:")
    #print(opening_costs_depots)
            
    line = next(file)
            
            
    opening_cost_route = int(line.strip())
                
    line = next(file)
            
    #print("opening cost route:")
    #print(opening_cost_route)
            
    line = next(file)
    zero_or_not = int(line.strip())
            
    #print("zero o uno: " + zero_or_not)
    return [n_customers, n_available_depots, v_capacity, opening_cost_route, zero_or_not]
    
def distance_cost(a1, a2, b1, b2):
    a1 = int(a1)
    a2 = int(a2)
    b1 = int(b1)
    b2 = int(b2)
    x1 = a1-b1
    x2 = a2-b2
    x1 = pow(x1, 2)
    x2 = pow(x2, 2)
    return math.sqrt(x1 + x2)

In [ ]:
file_path = "Inst1"
for filename in sorted(os.listdir(file_path), key=len):
    print("\nSiguiente archivo\n")
    f = os.path.join(file_path, filename)
    # checking if it is a file
    if os.path.isfile(f):
        
        #Contenedores de info inicializados en nada
        n_customers = 0
        n_available_depots = 0
        coordinates_depots = []
        coordinates_customers = []
        v_capacity = 0
        capacities_depots = []
        customers_demands = []
        opening_costs_depots = []
        opening_cost_route = 0
        zero_or_not = 1
        
        #Acá se toma la info de los archivos .dat
        with open(f, 'r') as file:
            lista_var = []
            lista_var = fill_variables(file, n_customers, n_available_depots, coordinates_depots, coordinates_customers, 
                                       v_capacity, capacities_depots, customers_demands, opening_costs_depots, 
                                       opening_cost_route, zero_or_not)
            n_customers = lista_var[0]
            n_available_depots = lista_var[1]
            v_capacity = lista_var[2]
            opening_cost_route = lista_var[3]
            zero_or_not = lista_var[4]
        
        #Empezar operaciones acá
        #distancias_d2d = []  VIAJE ENTRE 2 DEPOTS NO PERMITIDO
        distancias_d2c = []
        distancias_c2c = []
        distancias_c2d = []
        distancias = []
        
        #Acá se consiguen los c_ij (costos de caminos)
        for i in coordinates_depots:
            '''for j in coordinates_depots:
                if j==i:
                    continue
                distancias.append(distance_cost(i[0], i[1], j[0], j[1]))
                distancias_d2d.append(distance_cost(i[0], i[1], j[0], j[1]))'''
            for j in coordinates_customers:
                distancias.append(distance_cost(i[0], i[1], j[0], j[1]))
                distancias_d2c.append(distance_cost(i[0], i[1], j[0], j[1]))
        for i in coordinates_customers:
            for j in coordinates_depots:
                distancias.append(distance_cost(i[0], i[1], j[0], j[1]))
                distancias_c2d.append(distance_cost(i[0], i[1], j[0], j[1]))
                

        for i in range(len(coordinates_customers)):
            for j in range(len(coordinates_customers)):
                b = coordinates_customers[i]
                c = coordinates_customers[j]
                if j==i:
                    continue
                distancias.append(distance_cost(b[0], b[1], c[0], c[1]))
                distancias_c2c.append(distance_cost(b[0], b[1], c[0], c[1]))
                
        #MODELO
        mdl = Model(name="MDRP")
        x = mdl.binary_var_list(len(distancias), name="x")
        
        #print(len(distancias))
        #C_ij * X_ij
        ab = [distancias[i]*x[i] for i in range(len(distancias))]
        mdl.minimize(mdl.sum(ab))

        #X_dj = yd con yd = 1
        for i in range(n_available_depots):
            offset = i*n_customers
            xdj = []
            for j in range(offset, offset+n_customers):
                xdj.append(x[j])
            mdl.add_constraint(sum(xdj) == 1)
            
        #X_jd = yd con yd = 1
        for i in range(n_available_depots):
            xjd = []
            for j in range(n_customers):
                offset = j*n_available_depots + n_available_depots*n_customers
                #print(str(offset) + " vs " + str(len(distancias)) + " vs " + str(len(x)))
                xjd.append(x[offset])
            mdl.add_constraint(sum(xjd) == 1)

            
        #X_ij = 1
        for i in range(n_customers):
            aux = []
            for j in range(n_available_depots):
                aux.append(x[n_available_depots*i + j + len(distancias_d2c)])
            for j in range(n_customers-1):
                aux.append(x[(n_customers-1)*i + j + len(distancias_d2c) + len(distancias_c2d)])
            mdl.add_constraint(sum(aux) == 1)
        
        #X_ji = 1
        for i in range(n_customers):
            aux = []
            for j in range(n_available_depots):
                aux.append(x[j*n_available_depots + i])
            for j in range(n_customers):
                if(j == i):
                    continue
                aux.append(x[j*(n_customers-1) + i + len(distancias_d2c) + len(distancias_c2d)])
            mdl.add_constraint(sum(aux) == 1)

        
        # DL para F-MDRP
        u = mdl.continuous_var_list(n_customers, name="u")
        for i in range(n_customers):
            offset_ij = (n_customers-1)*i + len(distancias_d2c) + len(distancias_c2d)
            for j in range(n_customers):
                if i==j:
                    continue
                offset_ji = (n_customers-1)*j + len(distancias_d2c) + len(distancias_c2d)
                #print(str(j + offset_ij) + " y " + str(i + offset_ji) + " contra " + str(len(x)))
                mdl.add_constraint(u[i] - u[j] + (n_customers-n_available_depots+1)*x[j + offset_ij] + (n_customers-n_available_depots-1)*x[i + offset_ji]  <= n_customers - n_available_depots)
        
        
        # NODE-DL para MDRP
        k = mdl.continuous_var_list(n_customers, name="k")
        
        #Constraint 20
        for i in range(n_customers):
            for j in range(n_available_depots):
                mdl.add_constraint(k[i] + (n_available_depots - (j+1))*x[j*n_customers + i] <= n_available_depots)
                
        #Constraint 21
        for i in range(n_customers):
            for j in range(n_available_depots):
                mdl.add_constraint(k[i] + (n_available_depots - (j+1))*x[i*n_available_depots + j + len(distancias_d2c)] <= n_available_depots)
        
        #Constraint 22
        for i in range(n_customers):
            for j in range(n_available_depots):
                mdl.add_constraint(1 + ((j+1)-1)*x[j*n_customers + i] <= k[i])
                
        #Constraint 23
        for i in range(n_customers):
            for j in range(n_available_depots):
                mdl.add_constraint(1 + ((j+1) - 1)*x[i*n_available_depots + j + len(distancias_d2c)] <= k[i])
        
        #Constraint 24
        for i in range(n_customers):
            offset_ij = (n_customers-1)*i + len(distancias_d2c) + len(distancias_c2d)
            for j in range(n_customers):
                if i==j:
                    continue
                offset_ji = (n_customers-1)*j + len(distancias_d2c) + len(distancias_c2d)
                #print(str(j + offset_ij) + " y " + str(i + offset_ji) + " contra " + str(len(x)))
                mdl.add_constraint(k[i] - k[j] <= (n_available_depots - 1)*(1 - x[j + offset_ij] - x[i + offset_ji]))
        
        mdl.print_information()
        solution = mdl.solve()
        
        # Verificar si se encontró una solución
        if solution:
            # Extraer el valor óptimo de la función objetivo
            print(f"Valor óptimo de la función objetivo: {solution.objective_value}")
        else:
            print("No se encontró solución óptima.")
            
        solution.display()


Siguiente archivo

Número de customers: 20
Número de depots disponibles: 5
201 y 219 contra 580
202 y 238 contra 580
203 y 257 contra 580
204 y 276 contra 580
205 y 295 contra 580
206 y 314 contra 580
207 y 333 contra 580
208 y 352 contra 580
209 y 371 contra 580
210 y 390 contra 580
211 y 409 contra 580
212 y 428 contra 580
213 y 447 contra 580
214 y 466 contra 580
215 y 485 contra 580
216 y 504 contra 580
217 y 523 contra 580
218 y 542 contra 580
219 y 561 contra 580
219 y 201 contra 580
221 y 239 contra 580
222 y 258 contra 580
223 y 277 contra 580
224 y 296 contra 580
225 y 315 contra 580
226 y 334 contra 580
227 y 353 contra 580
228 y 372 contra 580
229 y 391 contra 580
230 y 410 contra 580
231 y 429 contra 580
232 y 448 contra 580
233 y 467 contra 580
234 y 486 contra 580
235 y 505 contra 580
236 y 524 contra 580
237 y 543 contra 580
238 y 562 contra 580
238 y 202 contra 580
239 y 221 contra 580
241 y 259 contra 580
242 y 278 contra 580
243 y 297 contra 580
244 y 316 contra 580


x_145 = 1
x_193 = 1
x_210 = 1
x_226 = 1
x_240 = 1
x_301 = 1
x_316 = 1
x_367 = 1
x_395 = 1
x_409 = 1
x_444 = 1
x_458 = 1
x_480 = 1
x_499 = 1
x_505 = 1
x_535 = 1
x_578 = 1
u_0 = 2.000
u_1 = 8.000
u_2 = 6.000
u_5 = 4.000
u_6 = 5.000
u_7 = 9.000
u_10 = 3.000
u_11 = 1.000
u_12 = 6.000
u_14 = 2.000
u_15 = 1.000
u_16 = 7.000
u_17 = 5.000
u_19 = 4.000
k_0 = 3.000
k_1 = 3.000
k_2 = 3.000
k_3 = 2.000
k_4 = 2.000
k_5 = 3.000
k_6 = 3.000
k_7 = 3.000
k_8 = 5.000
k_9 = 1.000
k_10 = 3.000
k_11 = 3.000
k_12 = 3.000
k_13 = 3.000
k_14 = 5.000
k_15 = 5.000
k_16 = 3.000
k_17 = 3.000
k_18 = 4.000
k_19 = 3.000

Siguiente archivo

Número de customers: 20
Número de depots disponibles: 5
201 y 219 contra 580
202 y 238 contra 580
203 y 257 contra 580
204 y 276 contra 580
205 y 295 contra 580
206 y 314 contra 580
207 y 333 contra 580
208 y 352 contra 580
209 y 371 contra 580
210 y 390 contra 580
211 y 409 contra 580
212 y 428 contra 580
213 y 447 contra 580
214 y 466 contra 580
215 y 485 contra 580
216 y 504 con

Valor óptimo de la función objetivo: 145.8021044558038
solution for: MDRP
objective: 145.802
status: OPTIMAL_SOLUTION(2)
x_9 = 1
x_28 = 1
x_58 = 1
x_67 = 1
x_95 = 1
x_100 = 1
x_138 = 1
x_141 = 1
x_154 = 1
x_226 = 1
x_238 = 1
x_259 = 1
x_282 = 1
x_298 = 1
x_315 = 1
x_376 = 1
x_426 = 1
x_442 = 1
x_462 = 1
x_482 = 1
x_495 = 1
x_520 = 1
x_540 = 1
x_553 = 1
x_573 = 1
u_0 = 4.000
u_1 = 2.000
u_2 = 3.000
u_3 = 2.000
u_5 = 1.000
u_6 = 1.000
u_7 = 3.000
u_10 = 2.000
u_11 = 1.000
u_12 = 4.000
u_14 = 5.000
u_15 = 1.000
u_16 = 6.000
u_17 = 2.000
u_19 = 3.000
k_0 = 1.000
k_1 = 4.000
k_2 = 1.000
k_3 = 1.000
k_4 = 4.000
k_5 = 1.000
k_6 = 4.000
k_7 = 4.000
k_8 = 2.000
k_9 = 1.000
k_10 = 5.000
k_11 = 3.000
k_12 = 4.000
k_13 = 5.000
k_14 = 4.000
k_15 = 5.000
k_16 = 4.000
k_17 = 3.000
k_18 = 3.000
k_19 = 4.000

Siguiente archivo

Número de customers: 50
Número de depots disponibles: 5
501 y 549 contra 2950
502 y 598 contra 2950
503 y 647 contra 2950
504 y 696 contra 2950
505 y 745 contra 2950
506 y 794 c

1359 y 1791 contra 2950
1360 y 1840 contra 2950
1361 y 1889 contra 2950
1362 y 1938 contra 2950
1363 y 1987 contra 2950
1364 y 2036 contra 2950
1365 y 2085 contra 2950
1366 y 2134 contra 2950
1367 y 2183 contra 2950
1368 y 2232 contra 2950
1369 y 2281 contra 2950
1370 y 2330 contra 2950
1371 y 2379 contra 2950
1372 y 2428 contra 2950
1373 y 2477 contra 2950
1374 y 2526 contra 2950
1375 y 2575 contra 2950
1376 y 2624 contra 2950
1377 y 2673 contra 2950
1378 y 2722 contra 2950
1379 y 2771 contra 2950
1380 y 2820 contra 2950
1381 y 2869 contra 2950
1382 y 2918 contra 2950
1382 y 518 contra 2950
1383 y 567 contra 2950
1384 y 616 contra 2950
1385 y 665 contra 2950
1386 y 714 contra 2950
1387 y 763 contra 2950
1388 y 812 contra 2950
1389 y 861 contra 2950
1390 y 910 contra 2950
1391 y 959 contra 2950
1392 y 1008 contra 2950
1393 y 1057 contra 2950
1394 y 1106 contra 2950
1395 y 1155 contra 2950
1396 y 1204 contra 2950
1397 y 1253 contra 2950
1398 y 1302 contra 2950
1399 y 1351 contra 2950
14

1733 y 917 contra 2950
1734 y 966 contra 2950
1735 y 1015 contra 2950
1736 y 1064 contra 2950
1737 y 1113 contra 2950
1738 y 1162 contra 2950
1739 y 1211 contra 2950
1740 y 1260 contra 2950
1741 y 1309 contra 2950
1742 y 1358 contra 2950
1743 y 1407 contra 2950
1744 y 1456 contra 2950
1745 y 1505 contra 2950
1746 y 1554 contra 2950
1747 y 1603 contra 2950
1748 y 1652 contra 2950
1749 y 1701 contra 2950
1751 y 1799 contra 2950
1752 y 1848 contra 2950
1753 y 1897 contra 2950
1754 y 1946 contra 2950
1755 y 1995 contra 2950
1756 y 2044 contra 2950
1757 y 2093 contra 2950
1758 y 2142 contra 2950
1759 y 2191 contra 2950
1760 y 2240 contra 2950
1761 y 2289 contra 2950
1762 y 2338 contra 2950
1763 y 2387 contra 2950
1764 y 2436 contra 2950
1765 y 2485 contra 2950
1766 y 2534 contra 2950
1767 y 2583 contra 2950
1768 y 2632 contra 2950
1769 y 2681 contra 2950
1770 y 2730 contra 2950
1771 y 2779 contra 2950
1772 y 2828 contra 2950
1773 y 2877 contra 2950
1774 y 2926 contra 2950
1774 y 526 contra 

2235 y 1515 contra 2950
2236 y 1564 contra 2950
2237 y 1613 contra 2950
2238 y 1662 contra 2950
2239 y 1711 contra 2950
2240 y 1760 contra 2950
2241 y 1809 contra 2950
2242 y 1858 contra 2950
2243 y 1907 contra 2950
2244 y 1956 contra 2950
2245 y 2005 contra 2950
2246 y 2054 contra 2950
2247 y 2103 contra 2950
2248 y 2152 contra 2950
2249 y 2201 contra 2950
2251 y 2299 contra 2950
2252 y 2348 contra 2950
2253 y 2397 contra 2950
2254 y 2446 contra 2950
2255 y 2495 contra 2950
2256 y 2544 contra 2950
2257 y 2593 contra 2950
2258 y 2642 contra 2950
2259 y 2691 contra 2950
2260 y 2740 contra 2950
2261 y 2789 contra 2950
2262 y 2838 contra 2950
2263 y 2887 contra 2950
2264 y 2936 contra 2950
2264 y 536 contra 2950
2265 y 585 contra 2950
2266 y 634 contra 2950
2267 y 683 contra 2950
2268 y 732 contra 2950
2269 y 781 contra 2950
2270 y 830 contra 2950
2271 y 879 contra 2950
2272 y 928 contra 2950
2273 y 977 contra 2950
2274 y 1026 contra 2950
2275 y 1075 contra 2950
2276 y 1124 contra 2950
22

Model: MDRP
 - number of variables: 3050
   - binary=2950, integer=0, continuous=100
 - number of constraints: 6010
   - linear=6010
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP
